# Monomer Input Validation Testing
### Requirements:
A good input validation step should require that the input monomer smarts:
1. Be unambiguous. This means no "or" conditionals
2. Have all atoms with a single element contain:
    1. query connectivity (X?) with explicit connectivity that matches (even for H)
    2. formal charge (even if 0)
    3. NO other info/conditionals (send warning if this happens, but allow it). So if the User tries to send atomic mass of something that isn't supported, don't allow that
3. Require that all atoms:
    1. be connected
    2. have an atom map number
4. Require that all bonds have a single discrete bond order

Note that currently there is no restriction on where wild-type atoms exist and how many of them there are. Wildtype atoms are treated as a variable node not used for information/chemistry assignment, but them must be connected to all other nodes and have an atom number. 

### Approach
RDKit's internal atom query logic contains many of the tools necessary for validation, which is explored first. See the final few cells in this notebook for a working example for input validation and a few example cases. 


## 1. RDKit testing and query examples
From https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html:
Valid logical operators can be:
1. "not" -> `!`
2. "and" (high precedence) -> `&`
3. "or" -> `,`
4. "and" (low precedence) -> `;`

The only 2 operators that are allowed in this format are the 2 "and" operators. Rdkit's internal query logic represents this with a block of text and can be parsed:


In [2]:
from rdkit import Chem

In [3]:
smarts = "[C;X4;+0:1]-[C&X4&+0:3](-[C,X4,+0:2])(-[C&!X4&+0:4])-[*&X4&+0:5]"
qmol = Chem.MolFromSmarts(smarts)
qmol.GetAtomWithIdx(0).GetDegree()

1

In [4]:
def print_atom_info(a):
    print("=====================================================")
    print(f"          Atom with Idx {a.GetIdx()} and map num {a.GetAtomMapNum()}")
    print("=====================================================")
    print(f"degree is {a.GetDegree()}")
    print(f"formal charge is {a.GetFormalCharge()}")
    print(f"has query is {a.HasQuery()}")
    print(f"Query is desicribed as: \n{a.DescribeQuery()}")

In [5]:
print_atom_info(qmol.GetAtomWithIdx(0))

          Atom with Idx 0 and map num 1
degree is 1
formal charge is 0
has query is True
Query is desicribed as: 
AtomAnd
  AtomAnd
    AtomType 6 = val
    AtomTotalDegree 4 = val
  AtomFormalCharge 0 = val



In [6]:

print_atom_info(qmol.GetAtomWithIdx(1))

          Atom with Idx 1 and map num 3
degree is 4
formal charge is 0
has query is True
Query is desicribed as: 
AtomAnd
  AtomAnd
    AtomType 6 = val
    AtomTotalDegree 4 = val
  AtomFormalCharge 0 = val



In [7]:
print_atom_info(qmol.GetAtomWithIdx(2))

          Atom with Idx 2 and map num 2
degree is 1
formal charge is 0
has query is True
Query is desicribed as: 
AtomOr
  AtomOr
    AtomType 6 = val
    AtomTotalDegree 4 = val
  AtomFormalCharge 0 = val



In [8]:
print_atom_info(qmol.GetAtomWithIdx(3))

          Atom with Idx 3 and map num 4
degree is 1
formal charge is 0
has query is True
Query is desicribed as: 
AtomAnd
  AtomAnd
    AtomType 6 = val
    AtomTotalDegree 4 != val
  AtomFormalCharge 0 = val



In [9]:
print_atom_info(qmol.GetAtomWithIdx(4))

          Atom with Idx 4 and map num 5
degree is 1
formal charge is 0
has query is True
Query is desicribed as: 
AtomAnd
  AtomTotalDegree 4 = val
  AtomFormalCharge 0 = val



Smarts queries can then be enforced by looking for specific keywords in the query such as "!=", "AtomOr". "AtomTotalDegree" will be used (X? format) in all atoms to ensure that atoms with an explicit element are explicitly specified. 

## Setting queries
Setting queries is most usefull when handling transitions from user-inputted monomer formats to the internal json representation of monomers

In [10]:
a = qmol.GetAtomWithIdx(0)
print(a.GetSmarts())
new_a = Chem.AtomFromSmarts("[C&D3&+1:4]")
a.SetQuery(new_a)
print(a.GetSmarts())

[C&X4&+0:1]
[C&D3&+:1]


## 2. Working example

In [121]:


def is_valid_monomer(smarts):
    def is_connected(rdmol):
        if len(Chem.rdmolops.GetMolFrags(rdmol)) > 1:
            return False
        else:
            return True
    
    def is_valid_interior_atom(atom):
        atom_smarts = atom.GetSmarts()
        # ensure that no unsupported logical operators exist
        operators = r"!,;"
        if any(op in atom_smarts for op in operators):
            reason = f"{atom_smarts}: found an unsupported logical operator"
            return False, reason
        # ensure that no other unsupported atomic primitives are accepted
        unsupported_prims = r"@xXvrRhH*"
        if any(prim in atom_smarts for prim in unsupported_prims):
            reason = f"{atom_smarts}: found unsupported primitive"
            return False, reason
        # require that all elements are specified in #<n> format. This removes the H prototype edge case
        # Also require explicit connecitivity in D<n> format and explicit charge with either a + or -
        required_prims = r"[]#D:"
        if not all(prim in atom_smarts for prim in required_prims):
            reason = f"{atom_smarts}: required primitive not included"
            return False, reason
        charge_prims = r"-+"
        if not any(prim in atom_smarts for prim in charge_prims):
            reason = f"{atom_smarts}: no charge primitive on at least one atom"
            return False, reason
        if not atom.Match(atom):
            reason = f"{atom_smarts}: query does not match rdchem.Mol reading of the molecule (likely due to connectivity)"
            return False, reason
        return True, ""

    def is_valid_neighbor_atom(atom):
        atom_smarts = atom.GetSmarts()
        # ensure that no unsupported logical operators exist
        operators = r"!,;&"
        if any(op in atom_smarts for op in operators):
            reason = f"{atom_smarts}: found an unsupported logical operator"
            return False, reason
        # ensure that no other unsupported atomic primitives are accepted
        unsupported_prims = r"@xXDvrRhH"
        if any(prim in atom_smarts for prim in unsupported_prims):
            reason = f"{atom_smarts}: found unsupported primitive"
            return False, reason
        # require that atoms have a wildtype atom and label
        required_prims = r"[]*:"
        if not all(prim in atom_smarts for prim in required_prims):
            reason = f"{atom_smarts}: required primitive not included"
            return False, reason
        return True, ""
    
    def is_valid_bond(bond):
        valid_bond_types = [
                Chem.BondType.SINGLE,
                # Chem.BondType.AROMATIC, # no current support for aromatic bonds or 1.5 bonds
                Chem.BondType.DOUBLE,
                Chem.BondType.TRIPLE,
                Chem.BondType.QUADRUPLE,
                Chem.BondType.QUINTUPLE,
                Chem.BondType.HEXTUPLE,
            ]
        if bond.GetBondType() in valid_bond_types:
            return True
        else:
            return False

    qmol = Chem.MolFromSmarts(smarts)

    # check if graph is connected
    if not is_connected(qmol):
        reason = "not connected"
        return False, reason
    
    # check atom strings for required and unsupported primitives
    for atom in qmol.GetAtoms():
        if atom.GetAtomicNum() > 0:
            is_valid, reason = is_valid_interior_atom(atom)
            if not is_valid:
                return False, reason
        elif atom.GetAtomicNum() == 0 and "#" not in atom.GetSmarts():
            is_valid, reason = is_valid_neighbor_atom(atom)
            if not is_valid:
                return False, reason
        else:
            reason = "zero atomic number with # primitive (likely due to conditionals"
            return False, reason
        
    # ensure unique atom map numbers for each atom
    map_nums = [atom.GetAtomMapNum() for atom in qmol.GetAtoms()]
    unique_map_nums = set(map_nums)
    if len(map_nums) != len(unique_map_nums):
        reason = "non-unique atom map numbers"
        return False, reason
    
    # If all checks are passed, the smarts is valid
    return True, "all checks passed"


## 3. Examples

In [122]:
smarts = "[#6D4+0:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]"
is_valid_monomer(smarts)

[#6&D4&+0:1]   6   False
[#1&D1&+0:2]   1   False
[#1&D1&+0:3]   1   False
[#1&D1&+0:4]   1   False
[*:5]   0   True


(True, 'all checks passed')

In [123]:
import pandas as pd
changed_smarts = [
    "[#6D4+0:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "C(-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "[#6D4:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "[#6D4+0:1](-[#1D1:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "[#6D4+0:1](-[#1D1+0:2])(-[#1D1+0])(-[#1D1+0:4])-[*:5]",
    "[#6D4+0:1](-[#1D1+0:1])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "[#6,D4+0:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "[!#6D4+0:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "[#6D4-0:1](-[#1D1-0:2])(-[#1D1-0:3])(-[#1D1-0:4])-[*:5]",
    "[#6D3+0:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "[#6+0:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "[#6D4+0:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*D4:5]",
    "[CD4+0:1](-[CD1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]",
    "[#6&D4&+0:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:4])-[*:5]"
]
df = pd.DataFrame(columns=["smarts", "valid", "reason"])
for smarts in changed_smarts:
    is_valid, reason = is_valid_monomer(smarts)
    new_row = {"smarts": smarts, "valid": is_valid, "reason": reason}
    df.loc[len(df)] = new_row

print(df)
df.to_csv("junk.csv", sep="\t")

[#6&D4&+0:1]   6   False
[#1&D1&+0:2]   1   False
[#1&D1&+0:3]   1   False
[#1&D1&+0:4]   1   False
[*:5]   0   True
C   6   False
[#6&D4:1]   6   False
[#6&D4&+0:1]   6   False
[#1&D1:2]   1   False
[#6&D4&+0:1]   6   False
[#1&D1&+0:2]   1   False
[#1&D1&+0]   1   False
[#6&D4&+0:1]   6   False
[#1&D1&+0:1]   1   False
[#1&D1&+0:3]   1   False
[#1&D1&+0:4]   1   False
[*:5]   0   True
[#6,D4&+0:1]   0   True
[!#6&D4&+0:1]   0   True
[#6&D4&+0:1]   6   False
[#1&D1&+0:2]   1   False
[#1&D1&+0:3]   1   False
[#1&D1&+0:4]   1   False
[*:5]   0   True
[#6&D3&+0:1]   6   False
[#6&+0:1]   6   False
[#6&D4&+0:1]   6   False
[#1&D1&+0:2]   1   False
[#1&D1&+0:3]   1   False
[#1&D1&+0:4]   1   False
[D4:5]   0   True
[C&D4&+0:1]   6   False
[#6&D4&+0:1]   6   False
[#1&D1&+0:2]   1   False
[#1&D1&+0:3]   1   False
[#1&D1&+0:4]   1   False
[*:5]   0   True
                                               smarts  valid  \
0   [#6D4+0:1](-[#1D1+0:2])(-[#1D1+0:3])(-[#1D1+0:...   True   
1      C(-

In [74]:
atom = qmol.GetAtomWithIdx(0)
atom.GetExplicitValence()

3

In [75]:
dir(atom)

['ClearProp',
 'DescribeQuery',
 'ExpandQuery',
 'GetAtomMapNum',
 'GetAtomicNum',
 'GetBonds',
 'GetBoolProp',
 'GetChiralTag',
 'GetDegree',
 'GetDoubleProp',
 'GetExplicitBitVectProp',
 'GetExplicitValence',
 'GetFormalCharge',
 'GetHybridization',
 'GetIdx',
 'GetImplicitValence',
 'GetIntProp',
 'GetIsAromatic',
 'GetIsotope',
 'GetMass',
 'GetMonomerInfo',
 'GetNeighbors',
 'GetNoImplicit',
 'GetNumExplicitHs',
 'GetNumImplicitHs',
 'GetNumRadicalElectrons',
 'GetOwningMol',
 'GetPDBResidueInfo',
 'GetProp',
 'GetPropNames',
 'GetPropsAsDict',
 'GetQueryType',
 'GetSmarts',
 'GetSymbol',
 'GetTotalDegree',
 'GetTotalNumHs',
 'GetTotalValence',
 'GetUnsignedProp',
 'HasOwningMol',
 'HasProp',
 'HasQuery',
 'InvertChirality',
 'IsInRing',
 'IsInRingSize',
 'Match',
 'NeedsUpdatePropertyCache',
 'SetAtomMapNum',
 'SetAtomicNum',
 'SetBoolProp',
 'SetChiralTag',
 'SetDoubleProp',
 'SetExplicitBitVectProp',
 'SetFormalCharge',
 'SetHybridization',
 'SetIntProp',
 'SetIsAromatic',
 'Se

In [76]:
atom.GetImplicitValence()

0

In [77]:
[print(a) for a in atom.GetPropNames()]

molAtomMapNumber


[None]

In [78]:
atom.DescribeQuery()

'AtomAnd\n  AtomAnd\n    AtomAtomicNum 6 = val\n    AtomTotalDegree 3 = val\n  AtomFormalCharge 1 = val\n'

In [79]:
atom.Match(atom)

True

In [72]:
bond = qmol.GetBonds()[0]
print(bond.GetBondType())

SINGLE


In [81]:
type(atom)

rdkit.Chem.rdchem.QueryAtom